### Chat with a MySQL Database using Langchain and mixtral via Groq

#### Set up the database
- Download the Chinook database (https://github.com/lerocha/chinook-database)



In [1]:
# load database
from langchain_community.utilities import SQLDatabase
# db info
db_user = "root"
db_password = "<your password here>"
db_host = "localhost"
db_name = "Chinook"
mysql_uri=f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}"
db = SQLDatabase.from_uri(mysql_uri)

#### Creating a SQL chain
Use a LLM (mixtral here) to generate the SQL query based on the user's input.
We need to provide to the LLM :
- the database schema
- the user's input 


In [2]:
# define a function that returns the loaded database's schema

def get_schema(_):
    schema = db.get_table_info()
    return schema

In [3]:
# chat prompt template
from langchain_core.prompts import ChatPromptTemplate
template = """
Based on the table schema below, write a SQL query that would answer the user's question. 
<SCHEMA>{schema}</SCHEMA>
    
Write only the SQL query and nothing else. Do not wrap the SQL query in any other text, not even backticks.
    
For example:
Question: which 3 artists have the most tracks?
SQL Query: SELECT ArtistId, COUNT(*) as track_count FROM Track GROUP BY ArtistId ORDER BY track_count DESC LIMIT 3;
Question: Name 10 artists
SQL Query: SELECT Name FROM Artist LIMIT 10;
    
Your turn:
    
Question: {question}
SQL Query:
"""

chat_prompt = ChatPromptTemplate.from_template(template)

In [14]:

from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

# set environment varaible GROQ_API_KEY
load_dotenv() # load .env file to environment

# initialize a LLM model
llm = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0)

sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | chat_prompt
    | llm
    | StrOutputParser()
)



In [5]:
# let's test the SQL chain
user_question = 'how many albums are there in the database?'
sql_chain.invoke({"question": user_question})



'SELECT COUNT(*) FROM Album;'

#### Create the full chain
We have the SQL chain
Then, we can create the full chain that will allows us to chat with the DB using natural language.
- the SQL query generated by the SQL chain
- the response from the db to the generated SQL query 
- the user's input



In [6]:
template="""Based on the table schema below, question, sql query and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}
"""
prompt_response = ChatPromptTemplate.from_template(template)

In [7]:
# create a function (tool) that will run the SQL query and get the response from the db:
def run_query(query):
    return db.run(query)

In [8]:
# example
run_query("SELECT COUNT(*) FROM Album;")

'[(347,)]'

In [9]:
# full chain that will allow us to chat with the database using natural language.

full_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema=get_schema,
        response = lambda x: run_query(x["query"]),
    )
    | prompt_response
    | llm
    | StrOutputParser()
)

In [10]:
# let's test the full chain
user_question = "how many albums are there in the database?"
full_chain.invoke({"question": user_question})



'Based on the SQL response, there are 347 albums in the database.'